In [ ]:
%load_ext autoreload
%autoreload 2

import os, sys
from pathlib import Path

from matplotlib import pyplot as plt

from nerfstudio.utils.eval_utils import eval_setup

### Load configuration

In [ ]:
# Path to the config for a trained model
base_dir = Path('/workspace/fieldwork-data/birds/2024-02-06/environment/C0043')
model_dir = base_dir / 'rade-features'
mesh_dir = model_dir / 'mesh'
load_config = model_dir / '2025-07-25_040743/config.yml'

# Load the model
# config, pipeline, checkpoint_path, step = eval_setup(load_config)

### Example of meshing

While there are multiple methods provided, Open3DTSDFFusion works best. 

In [ ]:
from collab_splats.utils.mesh import Open3DTSDFFusion

mesh_dir = mesh_dir / "mesh_exports"

mesher = Open3DTSDFFusion(
    load_config, 
    depth_trunc=3.0, 
    depth_name="median_depth",
    output_dir=mesh_dir
)

mesher.main()

### Plot mesh

In [ ]:
import pyvista as pv

# Basic loading and plotting
def load_and_plot_ply(mesh_path: str, attribute: Optional[str, np.ndarray] = None):
    """
    Load a PLY mesh file and display it with basic visualization
    """
    # Load the PLY file
    mesh = pv.read(mesh_path)
    
    # Print basic information about the mesh
    print(f"Number of points: {mesh.n_points}")
    print(f"Number of cells: {mesh.n_cells}")
    print(f"Bounds: {mesh.bounds}")
    
    # Create a plotter and add the mesh
    plotter = pv.Plotter()

    if attribute is not None:
        if isinstance(attribute, str):
            attr = torch.load(attribute)
        else:
            attr = attribute
        plotter.add_mesh(mesh, scalars=attr, rgb=False)
    else:
        plotter.add_mesh(mesh, scalars="rgb", rgb=True)
    
    plotter.show_axes()
    plotter.show()

In [ ]:
def property_to_color(property_values, colormap='viridis'):
    """Convert property values to colors for visualization"""
    import matplotlib.cm as cm
    import matplotlib.colors as mcolors
    
    # Normalize values to [0,1]
    normalized = (property_values - property_values.min()) / (property_values.max() - property_values.min())
    
    # Apply colormap
    cmap = cm.get_cmap(colormap)
    colors = cmap(normalized)[:, :3]  # Remove alpha channel
    
    return colors

In [ ]:
similarity_cmap = property_to_color(similarity_map.detach().cpu().numpy())

In [ ]:
# mesh = pv.read('Open3dTSDFfusion_mesh.ply')
mesh.point_data['features'] = similarity_map


In [ ]:
# Create a plotter and add the mesh
plotter = pv.Plotter()
plotter.add_mesh(mesh, scalars='features', rgb=False)
plotter.show_axes()
plotter.show()

# mesh.plot(scalars='RGB', rgb=True)

In [ ]:
mesh_fn = './Open3dTSDFfusion_mesh.ply'
load_and_plot_ply(mesh_fn)